In [32]:
# 目標: X = P⋅X_w 3D轉2D
# 有了P剛好就有相機外參
# 最後就是依循2D相機在該view拍出來的相片 把對應3Dpoint cloud 做點雲上色
# 完成


import numpy as np
from PIL import Image

# 讀取圖像
image = Image.open('Santa.jpg')
# 讀取3D模型數據
data = np.loadtxt('Santa.xyz')

# 已知的2D圖像點和3D模型點
image_points = np.array([
    [1414, 161],
    [1221, 793],
    [1630, 813],
    [1284, 881],
    [1592, 889],
    [1442, 899], # NOSE
    [1186, 1303],
    [1669, 1298],
    [1194, 1510],
    [1670, 1509],
    [1437, 1761]
])
world_points = np.array([
    [0.298323, -13.327416, 68.322968, 1],
    [9.88592, -1.6465, 39.3496, 1],
    [-9.45138, -2.40189, 38.1438, 1],
    [6.46371, 8.72707, 37.4334, 1],     
    [-6.85018, 8.9755, 37.3274, 1],        
    [-0.37063, 9.19068, 37.2172, 1],# NOSE
    [10.6528, 14.2738, 20.1813, 1],
    [-10.0874, 13.9665, 20.2669, 1],
    [10.2553, 15.1937, 11.2776, 1],
    [-10.1453, 15.407, 11.5646, 1],
    [-0.29442, 16.0463, 0.316749,1]
])


#  Ap = 0 矩陣
# 矩陣大小如下
# A : 2n*12
# p : 12*1
# 至少要6個點
num_points = image_points.shape[0]
A = np.zeros((2 * num_points, 12))

# 建立2n*12的矩陣
for i in range(num_points):
    X, Y, Z, W = world_points[i]
    x, y = image_points[i]
    A[2 * i] = [X, Y, Z, W, 0, 0, 0, 0, -x * X, -x * Y, -x * Z, -x * W]
    A[2 * i + 1] = [0, 0, 0, 0, X, Y, Z, W, -y * X, -y * Y, -y * Z, -y * W]

# 使用奇異值分解（SVD）求解 P
U, S, Vt = np.linalg.svd(A)
P = Vt[-1].reshape(3, 4)  # Vt的最後一行 並且把它那一整行reshap成3x4矩陣 獲得解 P

ones = np.ones((data.shape[0], 1))
homogeneous_world_points = np.hstack((data[:, :3], ones))
#計算 從3D傳換到2D
projected_points = (P @ homogeneous_world_points.T).T
projected_points[:, :2] /= projected_points[:, [2]]  #獲得3D映射回2D的座標點了 恭喜


M = P[:, :3]  # P的前3x3部分
m_4 = P[:, 3]  # P的最後一列

# 計算相機中心 C
C = -np.linalg.inv(M).dot(m_4)

# 遍歷每個點，並計算投影後的顏色
output_data = []
for point, projected_point in zip(data, projected_points):
    x, y, z, nx, ny, nz = point
    px, py = projected_point[:2]

    # 從相機中心到點的向量
    view_vector = np.array([x, y, z]) - C
    # 讀取Stanta.xyz每個point cloud點的向量
    normal_vector = np.array([nx, ny, nz])

    # 檢查點是否在圖像範圍內且是否面向相機
    if np.dot(view_vector, normal_vector) < 0:
        # 那就從圖像中提取顏色
        color = image.getpixel((int(px), int(py)))
        r, g, b = color[:3]
        a = 255
    else:
        r, g, b, a = (0, 0, 0, 0)  # 對於不可見或超出範圍的點，選擇黑色 完全透明

    output_data.append([x, y, z, nx, ny, nz, r, g, b, a])

# 將結果保存到文件中
np.savetxt('m11202130.txt', np.array(output_data), fmt='%f %f %f %f %f %f %d %d %d %d')
print("Result is saved!")


Result is saved!


抱歉，我混淆了一些概念。你是對的，我們只有投影矩陣P，沒有單獨的旋轉矩陣R。

由於投影矩陣P包含了內部參數矩陣K和外部參數矩陣[R | t]的信息，我們可以從中提取出外部參數。具體地，投影矩陣P可以表示為：

```
P = K [R | t]
```

其中，K是內部參數矩陣，[R | t]是外部參數矩陣。

投影矩陣P是由相機內部的參數和相機位置以及方向共同決定的。因此，我們可以從投影矩陣P中直接提取相機的位置和方向，而不需要單獨的旋轉矩陣R和內部參數矩陣K。

相機的位置可以通過投影矩陣P的最後一列（即第4列）來表示，而相機的方向則包含在投影矩陣P的前三列中。

因此，我們可以使用以下步驟來計算相機的中心：

1. 提取投影矩陣P的前三列（表示相機的方向）和最後一列（表示相機的位置）。
2. 使用這些信息計算相機的中心。

這裡是計算相機中心的程式碼：

```python
M = P[:, :3]  # P的前3x3部分
m_4 = P[:, 3]  # P的最後一列

# 計算相機中心 C
C = -np.linalg.inv(M).dot(m_4)
```

希望這次解釋清楚了，如果還有疑問，請隨時告訴我。

In [35]:
# 產生一張2D標記座標的參考圖

import cv2
import numpy as np

# 读取照片
image = cv2.imread('Santa.jpg')

# 座标点
# image_points = np.array([
#     [1414, 161],
#     [1221, 793],
#     [1630, 813],
#     [1284, 881],
#     [1592, 889],
#     [1442, 899], # NOSE
#     [1186, 1303],
#     [1669, 1298],
#     [1194, 1510],
#     [1670, 1509],
#     [1437, 1761]
# ])

# 在图片上绘制点和坐标值
for point in image_points:
    x, y = point
    cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
    cv2.putText(image, f'({x},{y})', (x + 10, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

# 保存图片
cv2.imwrite('Santa_keypoint_2D.jpg', image)

# 显示图片
# cv2.imshow('Santa with keypoint', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


True